In [3]:
%run datasets.ipynb
%run trees_lib.ipynb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
#dataset = datasets_thurgau_7()
dataset = datasets_cities_200k()
#dataset = datasets_world_12M()


In [9]:
kdtree = build_kd_tree(key_id_tuples(dataset, ['latitude', 'longitude']), len(dataset))

  0%|          | 0/199725 [00:00<?, ?it/s]

In [10]:
import folium

m = folium.Map()
kw = {
    "fill": True,
    "weight": 1,
    "fill_opacity": 0.05
}

def walk_tree_bounds(node, bounds=None, depth=0):
    if node is None or depth > 10:
        return
    if bounds is None:
        bounds = [(-math.inf, math.inf)]*len(node.key)
    
    dimension_index = depth % len(bounds)
    key_dim = node.key[dimension_index]
    bounds_dim = bounds[dimension_index]

    # Split the bounds (hyper-)rectangle at the node's position along the dimension.
    left_bounds = copy.copy(bounds)
    left_bounds[dimension_index] = (bounds_dim[0], key_dim)
    right_bounds = copy.copy(bounds)
    right_bounds[dimension_index] = (key_dim, bounds_dim[1])

    # pre-order
    yield node, bounds, depth
    yield from walk_tree_bounds(node.left, left_bounds, depth+1)
    yield from walk_tree_bounds(node.right, right_bounds, depth+1)

def is_leaf(node):
    return node.left is None and node.right is None

def bounds_to_rectangle(bounds):
    # bounds are (lower, upper) tuples in each dimension
    # rectangles are two points (dim0, dim1, dim2) for lower and upper
    return [[dim[0] for dim in bounds], [dim[1] for dim in bounds]]

def draw_leaves(tree, map):
    limit = [[-90, 90], [-180, 180]]
    colors = ["red", "blue"]
    enclosure = [[math.inf, math.inf], [-math.inf, -math.inf]]
    groups = {}
    for node, bounds, depth in walk_tree_bounds(tree, bounds=limit):
        rect = bounds_to_rectangle(bounds)
        enclosure[0] = min(enclosure[0], rect[0])
        enclosure[1] = max(enclosure[1], rect[1])
        color = colors[depth % len(colors)]
        if not depth in groups:
            group = folium.FeatureGroup(f'Layer {depth}', show=False)
            group.add_to(m)
            groups[depth] = group
        else:
            group = groups[depth]
        
        if is_within_bounds(node.key, limit):
            folium.Rectangle(
                bounds=rect,
#                tooltip=dataset[node.value]['name'],
                color=color,
                **kw
            ).add_to(group)
            folium.CircleMarker(
                location=node.key,
                tooltip=dataset[node.value]['name'],
                color=color
            ).add_to(group)
    m.fit_bounds(enclosure)
    folium.LayerControl().add_to(m)


draw_leaves(kdtree, m)
m